In [3]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

start_date = "2024-01-05"
data_tsmc = yf.download("2330.TW", start=start_date, end=None, interval="1d")
data_nvidia = yf.download("NVDA", start=start_date, end=None, interval="1d")
data_amd = yf.download("AMD", start=start_date, end=None, interval="1d")
data_tsmc.columns = ['_'.join(col).strip() for col in data_tsmc.columns.to_flat_index()]
data_nvidia.columns = ['_'.join(col).strip() for col in data_nvidia.columns.to_flat_index()]
data_amd.columns = ['_'.join(col).strip() for col in data_amd.columns.to_flat_index()]
series_col = 'Close_2330.TW'
data = data_tsmc[[series_col]]

# Yesterday's return (as before)
data["Return_2330.TW(yesterday)"] = np.log(data[series_col] / data[series_col].shift(1)).shift(1)

# Yesterday's actual closing price
data["Close_2330.TW(yesterday)"] = data_tsmc[series_col].shift(1)

# Yesterday's volume
data["Volume_2330.TW(yesterday)"] = data_tsmc["Volume_2330.TW"].shift(1)


# ---- NVDA ----
data["Return_NVDA(yesterday)"] = np.log(data_nvidia["Close_NVDA"] / data_nvidia["Close_NVDA"].shift(1)).shift(1)
data["Close_NVDA(yesterday)"]  = data_nvidia["Close_NVDA"].shift(1)
data["Volume_NVDA(yesterday)"] = data_nvidia["Volume_NVDA"].shift(1)


# ---- AMD ----
data["Return_AMD(yesterday)"] = np.log(data_amd["Close_AMD"] / data_amd["Close_AMD"].shift(1)).shift(1)
data["Close_AMD(yesterday)"]  = data_amd["Close_AMD"].shift(1)
data["Volume_AMD(yesterday)"] = data_amd["Volume_AMD"].shift(1)
exog_cols = [col for col in data.columns if col not in["Date","Close_2330.TW"]]
print(data.columns)
data.to_csv("raw_data.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_2828\342969641.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_tsmc = yf.download("2330.TW", start=start_date, end=None, interval="1d")
[*********************100%***********************]  1 of 1 completed
C:\Users\user\AppData\Local\Temp\ipykernel_2828\342969641.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_nvidia = yf.download("NVDA", start=start_date, end=None, interval="1d")
[*********************100%***********************]  1 of 1 completed
C:\Users\user\AppData\Local\Temp\ipykernel_2828\342969641.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_amd = yf.download("AMD", start=start_date, end=None, interval="1d")
[*********************100%***********************]  1 of 1 completed

Index(['Close_2330.TW', 'Return_2330.TW(yesterday)',
       'Close_2330.TW(yesterday)', 'Volume_2330.TW(yesterday)',
       'Return_NVDA(yesterday)', 'Close_NVDA(yesterday)',
       'Volume_NVDA(yesterday)', 'Return_AMD(yesterday)',
       'Close_AMD(yesterday)', 'Volume_AMD(yesterday)'],
      dtype='object')



C:\Users\user\AppData\Local\Temp\ipykernel_2828\342969641.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Return_2330.TW(yesterday)"] = np.log(data[series_col] / data[series_col].shift(1)).shift(1)
C:\Users\user\AppData\Local\Temp\ipykernel_2828\342969641.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Close_2330.TW(yesterday)"] = data_tsmc[series_col].shift(1)
C:\Users\user\AppData\Local\Temp\ipykernel_2828\342969641.py:23: SettingWithCopyWarning: 
A value is trying to be set on a co

In [20]:
orig_dates = data.index.copy()
data = data.asfreq("D").ffill()
data = data.sort_index()
data.loc[~data.index.isin(orig_dates), "Volume_2330.TW(today)"] = 0
data['is_weekend'] = data.index.weekday >= 5


In [24]:
data['Close_2330.TW(tomorrow)'] = data['Close_2330.TW(today)'].shift(-1)
data = data.dropna()
data.to_csv("raw_data.csv")